# Introducción
Este proyecto utiliza la **API-FOOTBALL v3**, una API REST con información actualizada e histórica de más de 1100 ligas y copas (resultados, alineaciones, eventos, clasificaciones y estadísticas). El acceso se realiza mediante solicitudes HTTP autenticadas con API key, con filtros por fecha, liga, equipo o temporada.

En este notebook se documenta y ejecuta el **pipeline ETL orquestado con Prefect**, cuya lógica vive en `scripts/etl_fixtures.py`. Se muestra el **setup**, la **ejecución del flow**, y cómo **servir y monitorear** el flujo en **Prefect Cloud**. El pipeline persiste datos en **Delta Lake** (capas **Bronze → Silver → Gold**) y genera **exportables en CSV y Parquet**.

## 1. Configuración de entorno y rutas

In [1]:
# Este bloque asegura que exista la estructura mínima del datalake
# al correr el flujo desde el notebook (modo demo/documentación).
# Nota: en el script etl_fixtures.py las rutas se definen de nuevo
# para la ejecución real por CLI / Prefect.

import os

DATALAKE_ROOT = "data/etl_datalake"
BRONZE_FIXTURES = f"{DATALAKE_ROOT}/bronze/api_football/fixtures"
SILVER_FIXTURES = f"{DATALAKE_ROOT}/silver/api_football/fixtures"
GOLD_FIXTURES   = f"{DATALAKE_ROOT}/gold/api_football/fixtures"
EXPORTS_DIR     = f"{DATALAKE_ROOT}/exports"

for p in [BRONZE_FIXTURES, SILVER_FIXTURES, GOLD_FIXTURES, EXPORTS_DIR, "scripts"]:
    os.makedirs(p, exist_ok=True)

print("Estructura base creada/validada ✅")

Estructura base creada/validada ✅


## 2. Credenciales y configuración

In [2]:
# Se lee la configuración desde pipeline.conf
# Contiene dos secciones:
# [api-credentials] -> API-Football
# [prefect]         -> Prefect Cloud

from configparser import ConfigParser

parser = ConfigParser()
parser.read("pipeline.conf")

# --- API-Football ---
BASE_URL = parser["api-credentials"]["base_url"]
API_KEY  = parser["api-credentials"]["api_key"]

print("Config OK ✅")
print("BASE_URL:", BASE_URL)
print("API_KEY:  (oculto) ****")

Config OK ✅
BASE_URL: https://v3.football.api-sports.io
API_KEY:  (oculto) ****


## 3. Prefect: conexión y conceptos

Prefect permite **orquestar** un pipeline en pasos claros:

- **Flow** → el contenedor principal del proceso ETL.  
- **Task** → cada paso atómico dentro del flow (ej. extracción, transformación, carga).  
- **Serve / Deploy** → maneras de ejecutar y programar flows:  
  - *Serve*: correrlo como servicio local (útil en desarrollo).  
  - *Deploy*: registrarlo en Prefect Cloud para programarlo y monitorearlo.  

El login en Prefect Cloud se realiza *una sola vez desde la terminal*.
Aquí solo se muestra de manera ilustrativa.

In [3]:
# Prefect requiere login en Cloud, pero este paso se hace normalmente
# desde la terminal (una única vez por entorno).
# Aquí solo mostramos cómo se lee la API Key desde pipeline.conf.

prefect_credentials = parser["prefect"]
prefect_api_key = prefect_credentials["api_key"]

print("Prefect API Key cargada (oculta) ****")

# Login en Prefect Cloud (solo a modo ilustrativo; en práctica se hace una vez desde la terminal)
# !prefect cloud login -k {prefect_api_key}

Prefect API Key cargada (oculta) ****


In [4]:
import prefect
print("Prefect versión:", prefect.__version__)

Prefect versión: 2.20.9


## 4. Uso del flujo desde `scripts/etl_fixtures.py`

El flujo ETL está definido en `scripts/etl_fixtures.py`.  
En este notebook es posible:

- **4.1 Opción A — Corrida única (demo one-off):** ejecuta el flow una sola vez para validar la orquestación.  
- **4.2 Opción B — Servir el flow (opcional):** mantiene el flow activo como servicio local.

### 4.1 Opción A — Ejecutar una corrida orquestada (demo one-off)

In [5]:
import importlib

# Importa el script de orquestación
etl = importlib.import_module("scripts.etl_fixtures")

# Ejecuta el flujo ETL de forma local (modo recomendado dentro del Notebook)
etl.etl_parametrizable(endpoints=["fixtures"])

# Nota:
# El mismo flujo también puede ejecutarse desde la terminal:
#     python scripts/etl_fixtures.py
# o, dentro del Notebook:
#     !python scripts/etl_fixtures.py
# Todas estas opciones llaman al mismo flow definido en scripts/etl_fixtures.py.

06:38:51.924 | INFO    | prefect.engine - Created flow run 'funny-parrot' for flow 'etl-parametrizable'

06:38:51.929 | INFO    | Flow run 'funny-parrot' - View at https://app.prefect.cloud/account/1513bf29-3686-40b8-9dbf-c85ba6a6f8c0/workspace/377710aa-6343-48a1-a52d-8fd9be6fbba7/flow-runs/flow-run/068f20ea-bbfd-72f8-8000-a56b9c000dca

06:38:52.610 | INFO    | Flow run 'funny-parrot' - Created task run 'extract_data-0' for task 'extract_data'

06:38:52.610 | INFO    | Flow run 'funny-parrot' - Executing 'extract_data-0' immediately...

06:38:54.722 | INFO    | Task run 'get_data-fixtures' - Finished in state Completed()

...

06:39:00.116 | ERROR   | Task run 'to_silver-fixtures' - Encountered exception during execution:
Traceback (most recent call last):
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\engine.py", line 2169, in orchestrate_task_run
    result = await call.aresult()
             ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 327, in aresult
    return await asyncio.wrap_future(self.future)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 352, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\OneDrive\Documentos\Repositorios_notebooks\ETL_api_football\scripts\etl_fixtures.py", line 109, in transform_to_silver
    df_silver = format_datetime_columns(df_silver)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\OneDrive\Documentos\Repositorios_notebooks\ETL_api_football\etl_utils.py", line 246, in format_datetime_columns
    df["fixture_periods_second"] = pd.to_datetime(
                                   ^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\pandas\core\tools\datetimes.py", line 1067, in to_datetime
    values = convert_listlike(arg._values, format)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\pandas\core\tools\datetimes.py", line 407, in _convert_listlike_datetimes
    return _to_datetime_with_unit(arg, unit, name, utc, errors)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\pandas\core\tools\datetimes.py", line 512, in _to_datetime_with_unit
    arr = cast_from_unit_vectorized(arg, unit=unit)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "conversion.pyx", line 149, in pandas._libs.tslibs.conversion.cast_from_unit_vectorized
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\numpy\core\fromnumeric.py", line 3360, in round
    return _wrapfunc(a, 'round', decimals=decimals, out=out)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\numpy\core\fromnumeric.py", line 59, in _wrapfunc
    return bound(*args, **kwds)
           ^^^^^^^^^^^^^^^^^^^^
FloatingPointError: overflow encountered in multiply

06:39:00.443 | ERROR   | Task run 'to_silver-fixtures' - Finished in state Failed('Task run encountered an exception FloatingPointError: overflow encountered in multiply')

06:39:00.713 | ERROR   | Flow run 'funny-parrot' - Encountered exception during execution:
Traceback (most recent call last):
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\engine.py", line 894, in orchestrate_flow_run
    result = await flow_call.aresult()
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 327, in aresult
    return await asyncio.wrap_future(self.future)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 352, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\OneDrive\Documentos\Repositorios_notebooks\ETL_api_football\scripts\etl_fixtures.py", line 175, in etl_parametrizable
    transform_to_silver(endpoint)
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\tasks.py", line 712, in __call__
    return enter_task_run_engine(
           ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\engine.py", line 1469, in enter_task_run_engine
    return from_sync.wait_for_call_in_loop_thread(begin_run)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\_internal\concurrency\api.py", line 218, in wait_for_call_in_loop_thread
    return call.result()
           ^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 318, in result
    return self.future.result(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 179, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\concurrent\futures\_base.py", line 401, in __get_result
    raise self._exception
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 389, in _run_async
    result = await coro
             ^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\engine.py", line 1605, in get_task_call_return_value
    return await future._result()
           ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\futures.py", line 237, in _result
    return await final_state.result(raise_on_failure=raise_on_failure, fetch=True)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\states.py", line 91, in _get_state_result
    raise await get_state_exception(state)
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\engine.py", line 2169, in orchestrate_task_run
    result = await call.aresult()
             ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 327, in aresult
    return await asyncio.wrap_future(self.future)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 352, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\OneDrive\Documentos\Repositorios_notebooks\ETL_api_football\scripts\etl_fixtures.py", line 109, in transform_to_silver
    df_silver = format_datetime_columns(df_silver)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\OneDrive\Documentos\Repositorios_notebooks\ETL_api_football\etl_utils.py", line 246, in format_datetime_columns
    df["fi

06:39:01.039 | ERROR   | Flow run 'funny-parrot' - Finished in state Failed('Flow run encountered an exception. FloatingPointError: overflow encountered in multiply')

FloatingPointError: overflow encountered in multiply

### 4.2 Opción B — Servir el flow como agente local (opcional)  
Esta modalidad deja el flujo en ejecución continua y permite gestionarlo desde Prefect Cloud (ejecuciones, logs, programación).  
⚠️ La celda quedará ocupada hasta que la detengas manualmente desde tu entorno (Interrupt/Stop del notebook).
Por defecto está comentada para evitar que se ejecute por accidente.

In [ ]:
import importlib
etl = importlib.import_module("scripts.etl_fixtures")
# etl.etl_parametrizable.serve(name="ETL-Fixtures", endpoints=["fixtures"])

## 5. Monitoreo en Prefect

El monitoreo del flujo no se realiza desde el notebook, sino desde la **UI de Prefect Cloud**.  
Una vez que el flow se ejecuta (corrida única o servido como agente), es posible:

1. **Flows** → ver el listado de flows registrados (ejemplo: `ETL-Fixtures`).  
2. **Run history** → revisar el historial de ejecuciones con sus logs, tiempos y reintentos.  
3. **Blocks** → administrar la configuración de almacenamiento o infraestructura remota si se utiliza.  
4. **Schedules** → programar ejecuciones automáticas (requiere plan pago).